In [1]:
import numpy as np;
import matplotlib.pyplot as plt

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


In [18]:
def between_groups_distances(means):
    means2 = np.sum(np.square(means),axis = 0,keepdims=True)
    means1 = means2.T
    M = means1+means2 - 2. * np.dot(means.T,means)
    return M
    #return np.sum(M,axis = 0)
    
def in_group_distance(K,means,groups,D):
    wynik = np.empty(K)
    mapa = [(groups[id],id) for id in range(D.shape[1])]
    #print "grupy: ",groups
    #print D.shape[1]
    for i in range(K):   
        #macierz = np.take(D,[b for (a,b) in mapa if a == i] ,axis = 1)
        #M = np.dot(D,macierz)
        #print "dla i=",i,[b for (a,b) in mapa if a == i]
        M = D[:,[b for (a,b) in mapa if a == i]]
        #print M.shape
        M2 = np.sum(np.square(M),axis = 0,keepdims=True)
        M1 = M2.T
        res = M1+M2 - 2. * np.dot(M.T,M)
        wynik[i] = np.max(res)
    return wynik

def create_groups(D,K,means):
    D2 = np.sum(np.square(D),axis = 0,keepdims=True).T
    means2 = np.sum(np.square(means),axis = 0,keepdims=True)
    M = D2+means2 - 2. * np.dot(D.T,means)
    return np.argmin(M,axis = 1).T
    
def randomize(n,K):
    a = np.arange(n)
    np.random.shuffle(a)
    return a[:K]
def my_kmeans(K,D):
    iteracje = 0
    #k - liczba grup, D - macierz danych d x n, gdzie n to liczba elementow
    n = D.shape[1]
    old_means = []
    groups = []
    randomed = randomize(n,K)
    new_means = D[:,randomed]
    while(not np.array_equal(old_means,new_means)):
        iteracje += 1
        groups = create_groups(D,K,new_means)
        #print "grupy:",groups
        old_means = np.copy(new_means)
        macierz = np.take(np.eye(K),groups,axis = 0)
        #print macierz.shape,groups.shape
        new_means = np.dot(D,macierz)
        g_counts = np.maximum(np.sum(macierz,axis = 0),1)
        new_means /= g_counts
    print "Rozmiary grup: ",np.array(g_counts,dtype = int)
    print "Dystanse miedzy grupami: ",between_groups_distances(new_means)
    #print "Dystanse wewnatrz grupy: ",in_group_distance(K,new_means,groups,D)
    return groups, new_means

In [9]:
transakcje = [[int(id) for id in line.split()] for line in open("kosarak.dat")];
maks = np.max([np.max(wiersz) for wiersz in transakcje])


In [10]:
T = 1000
najczestsze = np.zeros(maks+1,dtype = int);
#najczestsze1 = np.zeros(maks+1,dtype = int);
for wiersz in transakcje:
        najczestsze[wiersz] += 1;
#najczestsze = 
#print sorted(najczestsze, reverse=True)
najczestsze_indeksy = np.argsort(najczestsze)
najczestsze = sorted(najczestsze, reverse=True)[-T:]
#print najczestsze_indeksy
naj_indeksy_T = np.flipud(najczestsze_indeksy[-T:]) #indeksy T najczestszych towarow malejaco
#print naj_indeksy_T

In [12]:
import time
t = time.time()
macierz = np.zeros((T,maks+1))
mapa = {naj_indeksy_T[i] : i for i in range(T)}
for wiersz in transakcje:
        naj_w_wierszu = [mapa[id] for id in wiersz if id in mapa]
        #print naj_w_wierszu
        #fromTMostPop = [order[id] for id in transactions[i] if id in order]
        for j in naj_w_wierszu:
            for k in wiersz:
                macierz[j][k] += 1 if j != k else 0
print(time.time() - t)

170.299120903


In [19]:
groups, means = my_kmeans(10,macierz)

Rozmiary grup:  [   97  2234   293     7   127     3     5 37837    27   641]
Dystanse miedzy grupami:  [[  1.19209290e-07   2.26551077e+08   6.85191433e+07   2.42839935e+09
    2.00679579e+08   3.40301487e+11   1.90887413e+10   2.65817441e+08
    3.25193402e+08   1.55003114e+08]
 [  2.26551077e+08  -1.39698386e-09   4.68718489e+07   3.65648877e+09
    1.00525117e+07   3.53427862e+11   2.28575738e+10   1.66968729e+06
    9.95729447e+08   6.98266482e+06]
 [  6.85191433e+07   4.68718489e+07   1.63912773e-07   3.07058524e+09
    4.10217518e+07   3.47736994e+11   2.11488681e+10   6.56725393e+07
    6.43292510e+08   1.78868550e+07]
 [  2.42839935e+09   3.65648877e+09   3.07058524e+09  -5.72204590e-06
    3.54774956e+09   2.91871422e+11   1.02780026e+10   3.78104121e+09
    1.20205315e+09   3.42800890e+09]
 [  2.00679579e+08   1.00525117e+07   4.10217518e+07   3.54774956e+09
    2.60770321e-08   3.52345069e+11   2.24931545e+10   1.49258987e+07
    9.31092445e+08   1.13711516e+07]
 [  3.40301

In [89]:
x = np.array([[1,2,3],[4,5,6],[7,8,9]])
print x[:,[1,2]]
g = [0,1,1]
m = [(g[id],id) for id in range(3)]
print m
#macierz = np.take(D,[b for (a,b) in mapa if a == i] ,axis = 1)
#M = np.dot(D,macierz)
y = [b for (a,b) in m if a == 1]
print y
print x[:,y]

[[2 3]
 [5 6]
 [8 9]]
[(0, 0), (1, 1), (1, 2)]
[1, 2]
[[2 3]
 [5 6]
 [8 9]]
